In [4]:
import torch
import sklearn
import numpy as np
from sklearn.manifold import TSNE
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [5]:
testsets = {
    "Progan" : ["progan_lsun"],
    "Biggan" : ["biggan_256","biggan_512"],
    "Gaugan" : ["gaugan"],
    "Stargan" : ["stargan"],
    "Stylegan 2": ["stylegan2_afhqv2_512x512",
                   "stylegan2_ffhq_256x256",
                   "stylegan2_ffhq_1024x1024",
                   "stylegan2_lsundog_256x256",
                   "whichfaceisreal"],
    "Stylegan 3" : ["stylegan3_r_afhqv2_512x512",
                    "stylegan3_r_ffhqu_256x256",
                    "stylegan3_r_ffhqu_1024x1024",
                    "stylegan3_t_afhqv2_512x512",
                    "stylegan3_t_ffhqu_256x256",
                    "stylegan3_t_ffhqu_1024x1024"],
    "EG3D" : ["eg3d"],
    "Taming Trans." : ["taming-transformers_class2image_ImageNet",
                             "taming-transformers_noise2image_FFHQ",
                             "taming-transformers_segm2image_valid"],
    "Dall-E Mini" : ["dalle-mini_valid"],
    "Dall-E 2" : ["dalle_2"],
    "Glide" : ["glide_text2img_valid"],
    "Latent Dif." : ["latent-diffusion_class2image_ImageNet",
                          "latent-diffusion_noise2image_FFHQ",
                          "latent-diffusion_noise2image_LSUNbedrooms",
                          "latent-diffusion_noise2image_LSUNchurches",
                          "latent-diffusion_text2img_valid"],
    "Stable Diff." : ["stable_diffusion_256"],
    "Stable Diff. v2" : ["stable_diffusion_512_50steps",
                             "stable_diffusion_512_100steps",
                             "stable_diffusion_512_200steps"],
    "Guided Dif." : ["guided-diffusion_class2image_ImageNet",  
                "guided-diffusion_noise2image_LSUNbedrooms",
                "guided-diffusion_noise2image_LSUNcats",
                "guided-diffusion_noise2image_LSUNhorses"],
    "DeepFakes" : ["deepfake"],
    "Pristine" : ["real_coco_valid", 
                  "real_imagenet_val", 
                  "real_ucid",
                  "ffhq",
                  "celebA",
                  ],
}

In [8]:
outfile = "out.txt"

csvs = {
    "Base0" : {
        "path" : "data/grag_accuracies.csv",
        "panda" : pd.read_csv("data/grag_accuracies.csv",header=0, index_col=0),
        "threshold" : "Grag2021_progan"
    },
    "Base1" : {
        "path" : "data/grag_accuracies.csv",
        "panda" : pd.read_csv("data/grag_accuracies.csv",header=0, index_col=0),
        "threshold" : "Grag2021_latent"
    },
    "Base0_RC" : {
        "path" : "data/grag_accuracies_RC.csv",
        "panda" : pd.read_csv("data/grag_accuracies_RC.csv",header=0, index_col=0),
        "threshold" : "Grag2021_progan"
    },
    "Base1_RC" : {
        "path" : "data/grag_accuracies_RC.csv",
        "panda" : pd.read_csv("data/grag_accuracies_RC.csv",header=0, index_col=0),
        "threshold" : "Grag2021_latent"
    },
    "1" : {
        "path" : "data/progan-trained/vitL14_progan_datacomp_720k_9nn_accuracies.csv",
        "panda" : pd.read_csv("data/progan-trained/vitL14_progan_datacomp_720k_9nn_accuracies.csv",header=0, index_col=0),
        "threshold" : "Acc."
    },
    "2" : {
        "path" : "data/progan-trained/vitL14_progan_openai_720k_9nn_accuracies.csv",
        "panda" : pd.read_csv("data/progan-trained/vitL14_progan_openai_720k_9nn_accuracies.csv",header=0, index_col=0),
        "threshold" : "Acc."
    },
    "3_progan" : {
        "path" : "/home/martin/Documents/MScProject/project_local/data/progan-trained/vitL14_progan_datacomp_720k_net_accuracies.csv",
        "panda" : pd.read_csv("/home/martin/Documents/MScProject/project_local/data/progan-trained/vitL14_progan_datacomp_720k_net_accuracies.csv", header=0, index_col=0),
        "threshold" : "0.85"
    },
    "4_progan" : {
        "path" : "/home/martin/Documents/MScProject/project_local/data/progan-trained/vitL14_progan_openai_720k_net_accuracies.csv",
        "panda" : pd.read_csv("/home/martin/Documents/MScProject/project_local/data/progan-trained/vitL14_progan_openai_720k_net_accuracies.csv",header=0, index_col=0),
        "threshold" : "0.30"
    },
    "5_progan" : {
        "path" : "data/progan-trained/vitH14_progan_laion_100k_net_accuracies.csv",
        "panda" : pd.read_csv("data/progan-trained/vitH14_progan_laion_100k_net_accuracies.csv",header=0, index_col=0),
        "threshold" : "0.55"
    },

    "3_stable" : {
        "path" : "data/stable-trained/vitL14_stable_100k_datacomp_net_accuracies.csv",
        "panda" : pd.read_csv("data/stable-trained/vitL14_stable_100k_datacomp_net_accuracies.csv", header=0, index_col=0),
        "threshold" : "0.85"
    },
    "4_stable" : {
        "path" : "data/stable-trained/vitL14_stable_100k_openai_net_accuracies.csv",
        "panda" : pd.read_csv("data/stable-trained/vitL14_stable_100k_openai_net_accuracies.csv",header=0, index_col=0),
        "threshold" : "0.10"
    },
    "5_stable" : {
        "path" : "data/stable-trained/vitH14_stable_100k_laion_net_accuracies.csv",
        "panda" : pd.read_csv("data/stable-trained/vitH14_stable_100k_laion_net_accuracies.csv",header=0, index_col=0),
        "threshold" : "0.55"
    },

    "3_fused" : {
        "path" : "data/fuse/progan+stable_vitL14_200k_datacomp_net_accuracy.csv",
        "panda" : pd.read_csv("data/fuse/progan+stable_vitL14_200k_datacomp_net_accuracy.csv", header=0, index_col=0),
        "threshold" : "0.95"
    },
    "4_fused" : {
        "path" : "data/fuse/progan+stable_vitL14_200k_openai_net_accuracy.csv",
        "panda" : pd.read_csv("data/fuse/progan+stable_vitL14_200k_openai_net_accuracy.csv",header=0, index_col=0),
        "threshold" : "0.35"
    },
    "5_fused" : {
        "path" : "data/fuse/progan+stable_vitH14_200k_laion_net_accuracy.csv",
        "panda" : pd.read_csv("data/fuse/progan+stable_vitH14_200k_laion_net_accuracy.csv",header=0, index_col=0),
        "threshold" : "0.90"
    },
    "4_progan_compressed" : {
        "path" : "data/compressed/progan_openai.csv",
        "panda" : pd.read_csv("data/compressed/progan_openai.csv",header=0, index_col=0),
        "threshold" : "0.10"
    },
    "4_stable_compressed" : {
        "path" : "data/compressed/stable_openai.csv",
        "panda" : pd.read_csv("data/compressed/stable_openai.csv",header=0, index_col=0),
        "threshold" : "0.10"
    },
    "4_fused_compressed" : {
        "path" : "data/compressed/fused_openai.csv",
        "panda" : pd.read_csv("data/compressed/fused_openai.csv",header=0, index_col=0),
        "threshold" : "0.20"
    },
    "5_progan_compressed" : {
        "path" : "ddata/compressed/progan_laion.csv",
        "panda" : pd.read_csv("data/compressed/progan_laion.csv",header=0, index_col=0),
        "threshold" : "0.15"
    },
    "5_stable_compressed" : {
        "path" : "data/compressed/stable_laion.csv",
        "panda" : pd.read_csv("data/compressed/stable_laion.csv",header=0, index_col=0),
        "threshold" : "0.30"
    },
    "5_fused_compressed" : {
        "path" : "data/compressed/fused_laion.csv",
        "panda" : pd.read_csv("data/compressed/fused_laion.csv",header=0, index_col=0),
        "threshold" : "0.10"
    }
    
}

to_create = [
    "Base0",
    "Base1",
    "Base0_RC",
    "Base1_RC",
    ]

f = open(outfile, "w")

for set in testsets:
    subsets = testsets[set]
    N = len(subsets)

    f.write(f"\multirow{{{N}}}{{*}}{{{set}}}")
    for subset in subsets:
        f.write(f" &  {subset.replace('_', ' ')}  ")
        for num in to_create:
            table = csvs[num]["panda"]
            threshold = csvs[num]["threshold"]
            acc = table.loc[subset, threshold]
            f.write(" & {acc:.2f}".format(acc=acc))
        f.write(" \\\\ \n")
    f.write("\hline \n")

f.close()
            

